In [ ]:
# check_model.ipynb

import joblib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_curve
)

# --- Load model and features ---
model = joblib.load("../app/model.pkl")
model_features = joblib.load("../app/model_features.pkl")

print("✅ Model loaded successfully")
print("Model type:", type(model))

# Identify model type
if hasattr(model, "coef_"):
    print("This looks like a Linear Model (e.g., Logistic Regression).")
elif hasattr(model, "feature_importances_"):
    print("This looks like a Tree-based Model (e.g., Random Forest, Gradient Boosting).")
else:
    print("Model type could not be determined directly.")

# --- Load dataset ---
df = pd.read_csv("../data/churn_clean.csv")

# Separate features and target
X = pd.get_dummies(df.drop(columns=["Churn"]))
X = X.reindex(columns=model_features, fill_value=0)
y = df["Churn"]

# --- Predictions ---
y_pred = model.predict(X)
y_proba = model.predict_proba(X)[:, 1]

# --- Evaluation Metrics ---
print("\n📊 Model Evaluation Results:")
print(f"Accuracy:  {accuracy_score(y, y_pred):.4f}")
print(f"Precision: {precision_score(y, y_pred):.4f}")
print(f"Recall:    {recall_score(y, y_pred):.4f}")
print(f"F1 Score:  {f1_score(y, y_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y, y_proba):.4f}")

# --- Detailed Classification Report ---
print("\n📄 Classification Report:")
print(classification_report(y, y_pred))

# --- Confusion Matrix ---
cm = confusion_matrix(y, y_pred)
print("🧾 Confusion Matrix:")
print(cm)

# =======================
# 📊 Visualization Section
# =======================

# --- Confusion Matrix Heatmap ---
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Churn", "Churn"], yticklabels=["No Churn", "Churn"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# --- ROC Curve ---
fpr, tpr, thresholds = roc_curve(y, y_proba)
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc_score(y, y_proba):.4f}")
plt.plot([0,1], [0,1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate (Recall)")
plt.title("ROC Curve")
plt.legend()
plt.show()
